# Local binary pattern - on artificial data
Came from 2020-09-04_Local_binary_pattern_with_covid
and came from 2020-09-29_Local_binary_pattern_with_covid
and came from 2020-10-11_Local_binary_pattern_with_covid

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
import skimage
import tifffile
import os
import pandas as pd

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if False:
    fileroot = '/home/ubuntu/Documents/'

colors_new=['#0072b2','#009e73','#d55e00', '#cc79a7','#f0e442','#56b4e9']

import sys
sys.path.append(fileroot+'VU_TEX/Ben_utilities/')
import useful_functions as uf 
import time

Loading Linux


In [15]:
import matplotlib
import matplotlib.patheffects as path_effects
from functools import partial
from skimage.feature import local_binary_pattern
from importlib import reload
from itertools import repeat
from datetime import datetime
from tqdm.notebook import trange

In [16]:
reload(uf)

<module 'useful_functions' from '/home/jovyan/VU_TEX/Ben_utilities/useful_functions.py'>

In [17]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [18]:
directory_out_top = fileroot + 'VU_TEX/playground_Ben/2022-02-17_UK_CIC/2022-09-06_Iridium_paper_analysis/2022-09-06b_LBP_full_image/'

In [19]:
directory_in_imgs = fileroot + '0ExternalData/2022-05-10_Newcastle_University_iridium_covid_UK_CIC_dataset/Channel_061_ID_minus_2/'
imgs = os.listdir(directory_in_imgs)
imgs.remove('Thumbs.db')
#imgs.sort(key=partial(uf.getvaluefromstringbest, variable='MS', ending='.', mydtype=int))
#imgs = ['2021-12-14_img3_scale_1.npy']
#imgs = [img for img in imgs if '.npy' in img]
imgs

['002_ROI004_ODAD-V003Ch_061_MS_035.tiff',
 '009_ROI006_ODAD_P021Ch_061_MS_209.tiff',
 '009_ROI008_ODAD-P035Ch_061_MS_213.tiff',
 '010_ROI006_EDAD-P079Ch_061_MS_244.tiff',
 '004_ROI008_EDAD-P030Ch_061_MS_080.tiff',
 '003_ROI002_ODAD-V007Ch_061_MS_053.tiff',
 '009_ROI007_ROI_ACFF-V012Ch_061_MS_212.tiff',
 '001_ROI002_OTHR-P005Ch_061_MS_008.tiff',
 '006_ROI008_BRON-P014Ch_061_MS_144.tiff',
 '002_ROI005_EDAD-V009Ch_061_MS_036.tiff',
 '009_ROI002_EDAD_P059Ch_061_MS_194.tiff',
 '012_ROI002_ACFF-V004Ch_061_MS_318.tiff',
 '009_ROI003_ROI_ACFF-V011Ch_061_MS_199.tiff',
 '006_ROI007_BRON-P021Ch_061_MS_142.tiff',
 '008_ROI007_BRON-P011Ch_061_MS_183.tiff',
 '006_ROI008_PRES-V011Ch_061_MS_146.tiff',
 '010_ROI007_EDAD-P073Ch_061_MS_249.tiff',
 '004_ROI010_EDAD-P031Ch_061_MS_084.tiff',
 '006_ROI002_ODAD-P027Ch_061_MS_129.tiff',
 '012_ROI003_EDAD-P009Ch_061_MS_323.tiff',
 '010_ROI004_EDAD-V034Ch_061_MS_235.tiff',
 '010_ROI001_EDAD-P069Ch_061_MS_220.tiff',
 '003_ROI008_PRES-P010Ch_061_MS_066.tiff',
 '0

In [20]:
radius_list = [round(1.499*1.327**(float(x))) for x in range(0, 15)]
print(radius_list); len(radius_list)

[1, 2, 3, 4, 5, 6, 8, 11, 14, 19, 25, 34, 45, 59, 79]


15

In [21]:
directory_metadata = fileroot + '0ExternalData/2022-05-10_Newcastle_University_iridium_covid_UK_CIC_dataset/'
filename_metadata = 'Cross_site_Technology_ROI_Summary_with_filenames.xlsx'
df2 = pd.read_excel(directory_metadata + filename_metadata, engine='openpyxl')
df2

,Unnamed: 0,Patient,"From ""Pure"" DAD patient?",Sample_id,Batch_no,Biobank,Pathology,Path_Class,IMC,MIBI,DSP-Protein,DSP-WTA,Filenames,MS,Check filename duplicates,MyID1
0,0,CVAIR-NCL 008 (DR613/20),0,001_ROI001_ACFF-V007,1,1,ACFF,7,1,NaN,NaN,NaN,001_ROI001_ACFF-V007Ch_061_MS_001.tiff,1,1,0
1,1,CVAIR-NCL 009 (DR634/20),0,001_ROI001_BRON-P006,1,1,BRON,6,1,NaN,NaN,NaN,001_ROI001_BRON-P006Ch_061_MS_002.tiff,2,1,1
2,2,MPM20-5,1,001_ROI001_EDAD-V012,1,2,EDAD,3,1,NaN,NaN,NaN,001_ROI001_EDAD-V012Ch_061_MS_003.tiff,3,1,2
3,3,CVAIR-NCL 007 (DR607/20),0,001_ROI001_OTHR-V004,1,1,OTHR,9,1,NaN,NaN,NaN,001_ROI001_OTHR-V004Ch_061_MS_004.tiff,4,1,3
4,4,CVAIR-NCL 008 (DR613/20),0,001_ROI002_ACFF-V008,1,1,ACFF,7,1,NaN,NaN,NaN,001_ROI002_ACFF-V008Ch_061_MS_005.tiff,5,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,346,CVAIR-NCL 005 (DR565/20 B-CEPA),1,012_ROI006_ODAD-V001,12,1,ODAD,5,1,NaN,NaN,NaN,012_ROI006_ODAD-V001Ch_061_MS_336.tiff,336,1,346
347,347,CVAIR-NCL 006 (DR598/20),1,012_ROI007_EDAD-P006,12,1,EDAD,3,1,NaN,NaN,NaN,012_ROI007_EDAD-P006Ch_061_MS_337.tiff,337,1,347
348,348,CVAIR-NCL 005 (DR565/20 B-CEPA),1,012_ROI007_ODAD-P001,12,1,ODAD,5,1,NaN,NaN,NaN,012_ROI007_ODAD-P001Ch_061_MS_338.tiff,338,1,348
349,349,CVAIR-NCL 006 (DR598/20),1,012_ROI008_PRES-P004,12,1,PRESPos,2,1,NaN,NaN,NaN,012_ROI008_PRES-P004Ch_061_MS_339.tiff,339,1,349


In [22]:
df_all = pd.DataFrame(imgs, columns=['Filenames'])
df_all

,Filenames
0,002_ROI004_ODAD-V003Ch_061_MS_035.tiff
1,009_ROI006_ODAD_P021Ch_061_MS_209.tiff
2,009_ROI008_ODAD-P035Ch_061_MS_213.tiff
3,010_ROI006_EDAD-P079Ch_061_MS_244.tiff
4,004_ROI008_EDAD-P030Ch_061_MS_080.tiff
...,...
346,010_ROI004_EDAD-V040Ch_061_MS_236.tiff
347,006_ROI004_BRON-P020Ch_061_MS_134.tiff
348,007_ROI001_EDAD-P045Ch_061_MS_154.tiff
349,006_ROI003_ODAD-P028Ch_061_MS_132.tiff


In [23]:
result = df_all.merge(df2, how='outer', indicator=True)
result[result['_merge'] != 'both']

,Filenames,Unnamed: 0,Patient,"From ""Pure"" DAD patient?",Sample_id,Batch_no,Biobank,Pathology,Path_Class,IMC,MIBI,DSP-Protein,DSP-WTA,MS,Check filename duplicates,MyID1,_merge


In [24]:
def get_dims_tiff(filepath):
    myimg = skimage.io.imread(filepath)
    return myimg.shape

df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all['dims'] = list(map(uf.get_dims_from_numpy_file, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
df_all['dims'] = list(map(get_dims_tiff, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
#df_all['dims_superpixel'] = list(map(uf.get_cropped_superpixel_dims, df_all['dims'], repeat(1)))

df_all = uf.create_df_cross_for_separate_dfs(df_all,
                                             pd.DataFrame(radius_list, columns=['radius']))

#df_all['n_points'] = [int(32/5*r) for r in df_all['radius']]
df_all['n_points'] = [math.ceil(2*math.pi*r) for r in df_all['radius']]
df_all['method'] = ['uniform']*len(df_all)

df_all.sort_values(['n_points', 'Filenames'], inplace=True)
df_all.reset_index(drop=True, inplace=True)

df_all

,Filenames,dims,radius,n_points,method
0,001_ROI001_ACFF-V007Ch_061_MS_001.tiff,"(1015, 1083)",1,7,uniform
1,001_ROI001_BRON-P006Ch_061_MS_002.tiff,"(1019, 1019)",1,7,uniform
2,001_ROI001_EDAD-V012Ch_061_MS_003.tiff,"(1034, 1068)",1,7,uniform
3,001_ROI001_OTHR-V004Ch_061_MS_004.tiff,"(1076, 1010)",1,7,uniform
4,001_ROI002_ACFF-V008Ch_061_MS_005.tiff,"(1072, 1087)",1,7,uniform
...,...,...,...,...,...
5260,012_ROI006_ODAD-V001Ch_061_MS_336.tiff,"(500, 500)",79,497,uniform
5261,012_ROI007_EDAD-P006Ch_061_MS_337.tiff,"(1000, 1000)",79,497,uniform
5262,012_ROI007_ODAD-P001Ch_061_MS_338.tiff,"(1000, 1000)",79,497,uniform
5263,012_ROI008_PRES-P004Ch_061_MS_339.tiff,"(500, 500)",79,497,uniform


In [25]:
def get_numpy_datatype_unsigned_int(largest_value):
    if largest_value <= 255:
        print('In lower 255')
        value = np.uint8
    elif largest_value <= 65535:
        value = np.uint16
    else:
        value = 0
    return value

mymax = np.max(df_all['n_points']); print(mymax)
mydtype = get_numpy_datatype_unsigned_int(mymax); print(mydtype)

497
<class 'numpy.uint16'>


In [26]:
for i in trange(len(df_all)):
#    print(i, '/', len(df_all))
    filename = df_all['Filenames'][i]
#    out_dims = df_all['dims_superpixel'][i]
    radius = df_all['radius'][i]
    n_points = df_all['n_points'][i]
    method = df_all['method'][i]

    start = datetime.now()
#    print(start)
    
    sub_directory_out = 'METHOD_'+ method + '_npoints_' + str(n_points) + '_radius_' + str(radius) + '/'
#    print(directory_out_top+sub_directory_out)
    if not os.path.exists(directory_out_top+sub_directory_out):
        os.mkdir(directory_out_top+sub_directory_out)
        
    if os.path.exists(directory_out_top+sub_directory_out+filename):
        print('Already exists: ', directory_out_top+sub_directory_out+filename)
        continue
    
    img = skimage.io.imread(directory_in_imgs + filename)
    lbp = local_binary_pattern(img, n_points, radius, method).astype(mydtype)
    np.save(directory_out_top+sub_directory_out+filename, lbp)
#    print(datetime.now() - start)

  0%|          | 0/5265 [00:00<?, ?it/s]

## 